# Multiple Class Pneumonia & TB

In [1]:
import tensorflow as tf


gpus = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3999)])
 

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
print(('Is your GPU available for use?\n{0}').format(
    'Yes, your GPU is available: True' if tf.test.is_gpu_available() == True else 'No, your GPU is NOT available: False'
))

print(('\nYour devices that are available:\n{0}').format(
    [device.name for device in tf.config.experimental.list_physical_devices()]
))

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
Is your GPU available for use?
Yes, your GPU is available: True

Your devices that are available:
['/physical_device:CPU:0', '/physical_device:XLA_CPU:0', '/physical_device:GPU:0', '/physical_device:XLA_GPU:0']


In [3]:
import cv2 as cv
import pickle
import random
import numpy as np
 

In [6]:
tb1_data = pickle.load( open( "C:/Users/uprz2/Downloads/cs330/data/tb1-256-grayscale", "rb" ) )
p_data = pickle.load( open( "C:/Users/uprz2/Downloads/cs330/data/pneumonia-256-grayscale", "rb" ) )
p_data = [(pair[0], 2 if pair[1] == 1 else 0) for pair in p_data] # change pneumonia true class to value 2

raw_data = tb1_data + p_data  # array of tuples( [x,y], 0|1|2 )

random.shuffle(raw_data)

fraction_train = 0.8
train_size = int(len(raw_data) * fraction_train)

x_train = []
y_train = []

for v in raw_data[0: train_size] :
    x_train.append(v[0])
    y_train.append(v[1])

x_train = np.array(x_train) 
y_train = np.array(y_train)    

x_test = []
y_test = []


for v in raw_data[train_size:]:
    x_test.append(v[0])
    y_test.append(v[1])
    

x_test = np.array(x_test) 
y_test = np.array(y_test)  


x_train, x_test = x_train / 255.0, x_test / 255.0



# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

print("Train size: {}, test size: {}".format(len(x_train), len(x_test)))

Train size: 5214, test size: 1304


In [7]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([Conv2D(32, 3, activation='relu'), 
                                    Flatten(), 
                                    Dense(64, activation='relu'), 
                                    Dense(3)])

model.compile(loss=[tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)],optimizer='adam',metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=8, epochs=30)

print("Evaluation: ")
model.evaluate(x_test, y_test)


                                

Epoch 1/30
652/652 [==============================] - 35s 53ms/step - loss: 1.4511 - accuracy: 0.8673
Epoch 2/30
652/652 [==============================] - 35s 54ms/step - loss: 0.1595 - accuracy: 0.9440
Epoch 3/30
652/652 [==============================] - 35s 54ms/step - loss: 0.0996 - accuracy: 0.9672
Epoch 4/30
652/652 [==============================] - 35s 54ms/step - loss: 0.0693 - accuracy: 0.9753
Epoch 5/30
652/652 [==============================] - 35s 54ms/step - loss: 0.0674 - accuracy: 0.9785
Epoch 6/30
652/652 [==============================] - 35s 54ms/step - loss: 0.0463 - accuracy: 0.9854
Epoch 7/30
652/652 [==============================] - 35s 54ms/step - loss: 0.0527 - accuracy: 0.9827
Epoch 8/30
652/652 [==============================] - 34s 52ms/step - loss: 0.0292 - accuracy: 0.9908
Epoch 9/30
652/652 [==============================] - 34s 53ms/step - loss: 0.0084 - accuracy: 0.9975
Epoch 10/30
652/652 [==============================] - 34s 53ms/step - loss: 0.044

[0.5509814620018005, 0.928680956363678]

In [8]:
# get stats on the validation set

 
predictions = model.predict(x_test)
predictions = tf.argmax(predictions, axis=1)

cm = tf.math.confusion_matrix(y_test, predictions, 3)

print(cm)

tf.Tensor(
[[341   8  31]
 [ 14  54   2]
 [ 37   1 816]], shape=(3, 3), dtype=int32)
